### Preparar ambiente

Primero descomentar las siguientes lineas e instalar PyAthena. Solo se hace la primera vez que se abre la instancia

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
!{sys.executable} -m pip install pytz

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Conectar con pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata'
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

#### Crear tablas por pais

In [4]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [5]:
pais = 'BR'

In [6]:
niv_adm = '4'

# Union de tablas usuarios

In [7]:
# Selecionando fecha de interes
fechas = pd.date_range('09/01/2020','09/30/2020')

In [8]:
# Generando matriz origen-destino historica
tabla_matrizOD_adms_historica = f'historico_mod_adm{niv_adm}_{pais}'
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    year = fecha.year
    mes = fecha.month
    dia = fecha.day
    codigo_year = str(year)
    codigo_mes = '0'+str(mes) if mes<10 else str(mes)
    codigo_dia = '0'+str(dia) if dia<10 else str(dia)
    codigo_fecha = codigo_year + codigo_mes + codigo_dia
    
    # Generando matriz origen-destino historica
    #===========================================
    tabla_matrizOD_adms_dia = f'matriz_od_adm{niv_adm}_{pais}_{codigo_fecha}'
    if i == 0 and (codigo_dia == '01' and codigo_mes == '02'):
        query = f'''
                    CREATE TABLE {tabla_matrizOD_adms_historica} AS
                    SELECT *
                    FROM {tabla_matrizOD_adms_dia}
                '''
    else:
        query = f'''
                    INSERT INTO {tabla_matrizOD_adms_historica} 
                    SELECT * 
                    FROM {tabla_matrizOD_adms_dia} 
                '''
    cursor.execute(query)
    tiempo = (time.time()-time_fecha)/60
    print(f"{codigo_fecha} insertada, {int(tiempo)} minutos")
print((time.time()-start_time)/60)    

20200901 insertada, 0 minutos
20200902 insertada, 0 minutos
20200903 insertada, 0 minutos
20200904 insertada, 0 minutos
20200905 insertada, 0 minutos
20200906 insertada, 0 minutos
20200907 insertada, 0 minutos
20200908 insertada, 0 minutos
20200909 insertada, 0 minutos
20200910 insertada, 0 minutos
20200911 insertada, 0 minutos
20200912 insertada, 0 minutos
20200913 insertada, 0 minutos
20200914 insertada, 0 minutos
20200915 insertada, 0 minutos
20200916 insertada, 0 minutos
20200917 insertada, 0 minutos
20200918 insertada, 0 minutos
20200919 insertada, 0 minutos
20200920 insertada, 0 minutos
20200921 insertada, 0 minutos
20200922 insertada, 0 minutos
20200923 insertada, 0 minutos
20200924 insertada, 0 minutos
20200925 insertada, 0 minutos
20200926 insertada, 0 minutos
20200927 insertada, 0 minutos
20200928 insertada, 0 minutos
20200929 insertada, 0 minutos
20200930 insertada, 0 minutos
1.670025392373403


In [9]:
query = f'''MSCK REPAIR TABLE {tabla_matrizOD_adms_historica}'''
cursor.execute(query)